# Mountaincar Environment
* Eval can start anywhere from left to goal state, vel 0 (also training). They need 71 episodes
* Modify cartpole to only have two actions-> left and right. The magnitude of the actions are much larger in nfq paper
* Hint to goal, which sometimes makes the agent perform worse
* Group: the magnitude of the action
* Made the forces symmetric

In [1]:
import configargparse
import torch
import torch.optim as optim
import sys
sys.path.append('../')

from environments import MountainCarEnv, Continuous_MountainCarEnv
from models.agents import NFQAgent
from models.networks import NFQNetwork, ContrastiveNFQNetwork
from util import get_logger, close_logger, load_models, make_reproducible, save_models
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns
import tqdm

# Running experiments

## "Structureless Test"
* The dynamics of the systems are actually the same. Do any of the algorithms learn a difference?

In [ ]:
import json
from train_mountaincar import fqi, warm_start, transfer_learning
num_iter=15
perf_foreground = []
perf_background = []
for i in range(num_iter):
    print(str(i))
    perf_bg, perf_fg = fqi(epoch=1500, gravity=0.0025, verbose=True, is_contrastive=True, structureless=True)
    perf_foreground.append(perf_fg)
    perf_background.append(perf_bg)
sns.distplot(perf_foreground, label='Foreground Performance')
sns.distplot(perf_background, label='Background Performance')
plt.legend()
plt.xlabel("Average Reward Earned")
plt.title("Dynamics are the same in fg and bg environments")

0


  0%|          | 1/1501 [00:02<50:13,  2.01s/it]

Evaluation bg: [0, 0, 0, 0, 0] Evaluation fg: [0, 0, 0, 0, 0]


 20%|██        | 301/1501 [01:08<09:07,  2.19it/s]

Evaluation bg: [0, 0, 0, 0, 0] Evaluation fg: [0, 0, 0, 0, 0]


...

 20%|██        | 301/1501 [02:16<17:42,  1.13it/s]

Evaluation bg: [0, 0, 0, 0, 0] Evaluation fg: [0, 0, 0, 0, 0]


 40%|████      | 601/1501 [04:30<13:07,  1.14it/s]

Evaluation bg: [0, 0, 0, 0, 0] Evaluation fg: [0, 0, 0, 0, 0]


 60%|█████▉    | 900/1501 [06:42<04:33,  2.20it/s]

## "Performance when force left is different"
* We change the gravity on the foreground environments. 